Cyna Shirazinejad, 7/7/21

# outline of notebook 7:

* load data from cell lines:
* * AP2-tagRFP-T, tagGFP2-DNM2, ARPC3-HaloTag 
* * AP2-tagRFP-T, tagGFP2-DNM2, N-WASP-HaloTag 
* extract features from tracks
* * use existing feature scaler, decomposition axes, and mixture model to predict the identity of each new event
* merge the new data with existing tracks, features, and model cluster identities

# import all necessary Python modules

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import numpy as np
import sklearn.preprocessing as preprocessing
from sklearn.decomposition import PCA
import pickle
from scipy.fft import rfft, rfftfreq
from scipy import signal
unique_user_path_notebook = str(np.load('unique_user_path_notebook.npy'))
unique_user_saved_outputs = str(np.load('unique_user_saved_outputs.npy'))
unique_user_path_tracks = str(np.load('unique_user_path_tracks.npy'))
sys.path.append(unique_user_path_notebook+'/cmeAnalysisPostProcessingPythonScripts') # add custom Python scripts to the local path
import display_tracks
import merge_tools
import return_track_attributes
import generate_index_dictionary
import feature_extraction_with_buffer

# load dataframe from notebook 3 containing normal-pdf scaled features: PC's and GMM predicted clusters, and dataframe with cmeAnalysis labels

In [2]:
df_pcs_normal_scaled_with_gmm_cluster = pd.read_csv(unique_user_saved_outputs+'/dataframes/df_pcs_normal_scaled_with_gmm_cluster.zip')
df_merged_features = pd.read_csv(unique_user_saved_outputs+'/dataframes/df_merged_features.zip')
feature_units = np.load(unique_user_saved_outputs+'/dataframes/feature_units.npy')
index_DNM2positive = np.load(unique_user_saved_outputs+'/dataframes/cluster_dnm2_positive.npy')
number_of_track_splits = np.load(unique_user_saved_outputs+'/dataframes/number_of_track_splits.npy')
number_of_clusters = np.load(unique_user_saved_outputs+"/dataframes/number_of_clusters.npy")
best_fit_peak_params = np.load(unique_user_saved_outputs+'/dataframes/parameters_best_fit_peak_finding.npy')
ccp_predictions = np.load(unique_user_saved_outputs+'/dataframes/indices_ccp_predictions_' + \
                              'min_dist_'+str(best_fit_peak_params[0])+'_min_height_'+str(best_fit_peak_params[1])+ \
                              '_min_width_'+str(best_fit_peak_params[2])+ '.npy')

In [3]:
df_merged_features

,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_significant_dnm2,max_consecutive_significant_dnm2,fraction_significant_dnm2,fraction_peak_ap2,fraction_peak_dnm2,experiment_number,number_of_channels,date,cell_condition,cmeAnalysis_dynamin2_prediction
0,216.0,1796.284550,740.516756,7.203812,7.203812,2.630656,0.566262,0.725913,159.0,186.0,...,167.0,43.0,0.738938,0.703540,0.823009,0.0,2.0,200804.0,no-treatment,1.0
1,201.0,2215.532695,1505.433273,17.220726,14.507135,3.797526,0.424574,0.912671,111.0,107.0,...,115.0,89.0,0.545024,0.526066,0.507109,0.0,2.0,200804.0,no-treatment,1.0
2,201.0,864.976087,421.405691,17.621866,17.130473,3.472332,0.566588,0.776404,42.0,44.0,...,167.0,70.0,0.791469,0.199052,0.208531,0.0,2.0,200804.0,no-treatment,1.0
3,192.0,509.795166,356.302521,10.804211,13.292842,5.888569,0.428736,1.058177,44.0,190.0,...,112.0,57.0,0.554455,0.217822,0.940594,0.0,2.0,200804.0,no-treatment,1.0
4,188.0,1636.422386,883.606436,11.255090,7.793074,5.531006,0.415792,0.753372,174.0,169.0,...,169.0,80.0,0.853535,0.878788,0.853535,0.0,2.0,200804.0,no-treatment,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59234,2.0,28.623953,19.042675,1.879691,2.915253,2.389665,1.092336,1.354985,6.0,0.0,...,0.0,0.0,0.000000,0.500000,0.000000,7.0,2.0,200819.0,no-treatment,0.0
59235,2.0,15.164065,14.946216,7.447355,2.666432,6.454259,2.453558,1.923304,6.0,3.0,...,0.0,0.0,0.000000,0.500000,0.250000,7.0,2.0,200819.0,no-treatment,0.0
59236,2.0,21.879792,26.215772,4.384565,1.665404,4.318207,1.873770,2.418798,6.0,6.0,...,1.0,1.0,0.083333,0.500000,0.500000,7.0,2.0,200819.0,no-treatment,0.0
59237,2.0,20.236623,13.954595,1.079388,2.888644,3.897009,0.843483,2.105214,6.0,11.0,...,0.0,0.0,0.000000,0.500000,0.916667,7.0,2.0,200819.0,no-treatment,0.0


# load all valid tracks

In [4]:
len(set(df_merged_features['experiment_number']))

8

In [5]:
# load all valid tracks
merged_all_valid_tracks = np.load(unique_user_saved_outputs+'/dataframes/merged_all_valid_tracks_0.npy', allow_pickle=True)

for i in range(1,number_of_track_splits):

    merged_all_valid_tracks = np.concatenate((merged_all_valid_tracks,
                                             np.load(unique_user_saved_outputs+'/dataframes/merged_all_valid_tracks_'+str(i)+'.npy', allow_pickle=True)))

# load new ARPC3 imaging data, create a dataframe of merged features

In [6]:
# upload only AP2 and DNM2 data for now
all_tracks = [] # a list of all the track objects; each value is one experiment

# this cell is for the following experiment set: 200804_ADA3
tracks_200804ADA3Cell001_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200804_ADA3/split_channel_data/200804_ADA3_001/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200804ADA3Cell001AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200804ADA3Cell002_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200804_ADA3/split_channel_data/200804_ADA3_002/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200804ADA3Cell002AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200804ADA3Cell003_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200804_ADA3/split_channel_data/200804_ADA3_003/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200804ADA3Cell003AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200804ADA3Cell008_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200804_ADA3/split_channel_data/200804_ADA3_008/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200804ADA3Cell008AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200804ADA3Cell009_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200804_ADA3/split_channel_data/200804_ADA3_009/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200804ADA3Cell009AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')

all_tracks.append(tracks_200804ADA3Cell001_1s)
all_tracks.append(tracks_200804ADA3Cell002_1s)
all_tracks.append(tracks_200804ADA3Cell003_1s)
all_tracks.append(tracks_200804ADA3Cell008_1s)
all_tracks.append(tracks_200804ADA3Cell009_1s)

# this cell is for the following experiment set: 200819_ADA3
tracks_200819ADA3Cell001_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200819_ADA3/split_channel_data/200819_ADA3_001/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200819ADA3Cell001AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200819ADA3Cell002_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200819_ADA3/split_channel_data/200819_ADA3_002/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200819ADA3Cell002AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200819ADA3Cell003_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200819_ADA3/split_channel_data/200819_ADA3_003/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200819ADA3Cell003AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')

all_tracks.append(tracks_200819ADA3Cell001_1s)
all_tracks.append(tracks_200819ADA3Cell002_1s)
all_tracks.append(tracks_200819ADA3Cell003_1s)

# using previous imaging data (7/22) that did not have AP2/DNM2 cell line imaged at the same time

tracks_200722ADCell001_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200722_ADA3/split_channel_data/200722_ADA3_001/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200722ADA3Cell001AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200722ADCell002_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200722_ADA3/split_channel_data/200722_ADA3_002/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200722ADA3Cell002AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200722ADCell003_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200722_ADA3/split_channel_data/200722_ADA3_003/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200722ADA3Cell003AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200722ADCell004_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200722_ADA3/split_channel_data/200722_ADA3_004/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200722ADA3Cell004AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_200722ADCell005_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/200722_ADA3/split_channel_data/200722_ADA3_005/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/200722ADA3Cell005AP2DNMN2_1s/Ch1/Tracking/ProcessedTracks.mat')

all_tracks.append(tracks_200722ADCell001_1s)
all_tracks.append(tracks_200722ADCell002_1s)
all_tracks.append(tracks_200722ADCell003_1s)
all_tracks.append(tracks_200722ADCell004_1s)
all_tracks.append(tracks_200722ADCell005_1s)

In [7]:
len(all_tracks)

13

In [8]:
# extract valid tracks from 3 color cell line movies
valid_tracks_separate_experiments_3_color = [display_tracks.remove_tracks_by_criteria(track_set, track_category=[1]) for track_set in all_tracks]
# merge all valid tracks into one tracks array
merged_all_valid_tracks_3_color = merge_tools.merge_experiments(valid_tracks_separate_experiments_3_color,[list(range(len(track_set))) for track_set in valid_tracks_separate_experiments_3_color])
experiment_number_3_channel_label = [i 
                                     for i in range(len(set(df_merged_features['experiment_number'])), 
                                                    len(set(df_merged_features['experiment_number']))+len(valid_tracks_separate_experiments_3_color)) 
                                     for _ in range(len(valid_tracks_separate_experiments_3_color[i-len(set(df_merged_features['experiment_number']))]))]
# labels for the two days of imaging
date_of_experiment_3_channel = []
for i in range(len(experiment_number_3_channel_label)):
    
    if i < 5:
        
        date_of_experiment_3_channel.append(200804)
        
    elif i>=5 and i<8:
        
        date_of_experiment_3_channel.append(200819)
        
    else:
        
        date_of_experiment_3_channel.append(200722)

# labels for the number of total imaging channels for each event, but not actually including arpc3 data until later
# this is the number of fluorescently-edited genes in the cell line (AP2/DNM2 and now ARPC3)
number_of_channels_label = [3 for _ in range(len(experiment_number_3_channel_label))]

cell_condition = ['no_treatment' for i in range(len(experiment_number_3_channel_label))]

The number of tracks returned: 6265

The number of tracks returned: 7621

The number of tracks returned: 8396

The number of tracks returned: 7452

The number of tracks returned: 7724

The number of tracks returned: 7918

The number of tracks returned: 6663

The number of tracks returned: 6637

The number of tracks returned: 7668

The number of tracks returned: 7709

The number of tracks returned: 7431

The number of tracks returned: 7366

The number of tracks returned: 6275



# save all valid arpc3 tracks

In [9]:
for i in range(len(valid_tracks_separate_experiments_3_color)):
    
    np.save(unique_user_saved_outputs+"/dataframes/valid_arpc3_tracks_"+str(i), np.array(list(valid_tracks_separate_experiments_3_color[i])))

KeyboardInterrupt: 

In [ ]:
# extract the output of cmeAnalysis' predictions on whether a track is DNM2 positive or negative
significant_dynamin2_cmeAnalysis_prediction = []

# an index map for ProcessedTracks.mat attributes for 2 color tracking experiments from cmeAnalysis
index_dictionary = generate_index_dictionary.return_index_dictionary()

for track in merged_all_valid_tracks_3_color: # iterate through all tracks

    significant_dynamin2 = track[index_dictionary['index_significantSlave']][1]
    significant_dynamin2_cmeAnalysis_prediction.append(significant_dynamin2)

In [ ]:
print('total number of valid tracks: ' + str(len(merged_all_valid_tracks_3_color)))

In [ ]:
possible_track_features = np.load(unique_user_saved_outputs+'/dataframes/possible_track_features.npy')

In [ ]:
possible_track_features

In [ ]:
all_track_features_3_color = feature_extraction_with_buffer.TrackFeatures(merged_all_valid_tracks_3_color) # an instance of a to-be feature matrix of tracks
all_track_features_3_color.add_features(possible_track_features) # set the features to be extracted
all_track_features_3_color.extract_features() # extract all features
extracted_features_all_tracks_3_color = all_track_features_3_color.feature_matrix # feature matrix for all tracks

In [ ]:
# merge features with labels (experiment number, date, and number of channels)
extracted_features_all_tracks_3_color = np.array(extracted_features_all_tracks_3_color)

merged_features = np.concatenate((extracted_features_all_tracks_3_color,
                                  np.array(experiment_number_3_channel_label).reshape(extracted_features_all_tracks_3_color.shape[0],-1)), axis=-1)
merged_features = np.concatenate((merged_features,
                                  np.array(number_of_channels_label).reshape(merged_features.shape[0],-1)), axis=-1)
merged_features = np.concatenate((merged_features,
                                  np.array(date_of_experiment_3_channel).reshape(merged_features.shape[0],-1)), axis=-1)
merged_features = np.concatenate((merged_features,
                                  np.array(cell_condition).reshape(merged_features.shape[0],-1)), axis=-1)
merged_features = np.concatenate((merged_features,
                                  np.array(significant_dynamin2_cmeAnalysis_prediction).reshape(merged_features.shape[0],-1)), axis=-1)

In [ ]:
merged_features.shape

# use prefit scaler, PCA model, and GMM to fit new dataset to clusters

In [ ]:
with open(unique_user_saved_outputs+'/dataframes/normal_scaler_model', 'rb') as f:
    scaler = pickle.load(f)      
    
with open(unique_user_saved_outputs+'/dataframes/pca_model_fit', 'rb') as f:
    pca_model = pickle.load(f)              
    
with open(unique_user_saved_outputs+'/dataframes/gmm_trained', 'rb') as f:
    gmm_model = pickle.load(f)                

In [ ]:
scaled_features_new_data = scaler.transform(merged_features[:,:len(feature_units)]) # scale features to normal distribution, taking into account all previously scaled data
pcs_new_data = pca_model.transform(scaled_features_new_data) # find projections of newly scaled data on previous PC axes
gmm_predictions_new_data = gmm_model.predict(pcs_new_data) # find gmm cluster assignments using previously fit model

# run DNM2 positive events through smoothing and single-peak selection

In [ ]:
# get DNM2 positive events
dnm2_positive_events = np.array(list(merged_all_valid_tracks_3_color))[np.nonzero(gmm_predictions_new_data==index_DNM2positive)[0]]

In [ ]:
len(dnm2_positive_events)

In [ ]:
all_dnm2_signal = []

for i in range(len(dnm2_positive_events)): # stack all DNM2 intensities

    raw_dnm2_intensity = list(return_track_attributes.return_track_amplitude_no_buffer_channel(dnm2_positive_events,i,1))

    all_dnm2_signal.append(raw_dnm2_intensity)

In [ ]:
sos = signal.butter(4, 0.2, 'lp', fs=1, output='sos') # low-pass 4-th order Butterworth filter

filtered_amplitudes = [] # filtered DNM2 traces per track of interest

for i in range(len(all_dnm2_signal)):

    raw_intensity = all_dnm2_signal[i]
    # add zeros to end to account for phase shift of near-track-end peaks
    filtered_amplitudes.append(list(list(signal.sosfilt(sos, raw_intensity)) + [0, 0, 0, 0, 0])) 
    
current_param_outputs = [] # one-hot encoding of indices of tracks with a single peak (0: multiple peaks)


for i in range(len(filtered_amplitudes)): # iterate through all filtered amplitudes
    
    pvals_dnm2 = return_track_attributes.return_pvals_detection_no_buffer(dnm2_positive_events, i, 1)
    
    # measure whether there is 1 peak with the specified peak-finding parameters
    if len(signal.find_peaks(filtered_amplitudes[i], 
                             distance=best_fit_peak_params[0], 
                             height=best_fit_peak_params[1],
                             width=best_fit_peak_params[2])[0])==1 and len(np.where(np.array(pvals_dnm2)<0.01)[0])>0:

        current_param_outputs.append(1)

    else:

        current_param_outputs.append(0)

In [ ]:
len(np.where(np.array(current_param_outputs)==1)[0])

In [ ]:
data_add_pc_gmm_dataframe = np.hstack((pcs_new_data, gmm_predictions_new_data.reshape(pcs_new_data.shape[0], 1)))
df_new_incorporated_data_pcs_gmm_clusters = df_pcs_normal_scaled_with_gmm_cluster.copy()
df_new_incorporated_data_pcs_gmm_clusters = df_new_incorporated_data_pcs_gmm_clusters.append(pd.DataFrame(data_add_pc_gmm_dataframe, columns=df_pcs_normal_scaled_with_gmm_cluster.columns))

In [ ]:
df_new_incorporated_data_merged_features = df_merged_features.copy()
df_new_incorporated_data_merged_features = df_new_incorporated_data_merged_features.append(pd.DataFrame(merged_features, columns=df_merged_features.columns))

In [ ]:
# save the dataframe for ARPC3 tracking
compression_opts = dict(method='zip',
                        archive_name=unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_merged_features.csv')  

df_new_incorporated_data_merged_features.to_csv(unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_merged_features.zip', index=False,
                                                compression=compression_opts) 

In [ ]:
df_new_incorporated_data_merged_features

In [ ]:
compression_opts = dict(method='zip',
                        archive_name=unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_pcs_gmm_clusters.csv')  

df_new_incorporated_data_pcs_gmm_clusters.to_csv(unique_user_saved_outputs+'/dataframes/df_new_incorporated_data_pcs_gmm_clusters.zip', index=False,
                                                 compression=compression_opts) 

# upload hotspot predictions from previous data, then merge and save with newly incorporated predictions

In [ ]:
merged_ccp_predictions = np.array(list(ccp_predictions) + list(current_param_outputs))
np.save(unique_user_saved_outputs+'/dataframes/merged_ccp_predictions', merged_ccp_predictions)

In [ ]:
len(merged_ccp_predictions)

# merged all previous valid tracks with new valid tracks

In [ ]:
all_merged_valid_tracks = np.concatenate((merged_all_valid_tracks, np.array(list(merged_all_valid_tracks_3_color))))

In [ ]:
all_merged_valid_tracks.shape

In [ ]:
split_valid_tracks = np.array_split(np.array(list(all_merged_valid_tracks)),number_of_track_splits)

In [ ]:
# save each track array chunk
for i in range(len(split_valid_tracks)):

    np.save(unique_user_saved_outputs+"/dataframes/all_experiments_merged_all_valid_tracks_"+str(i), split_valid_tracks[i])

# load N-WASP imaging data, repeat pipeline for AP2/DNM2 and AP2/DNM2/ARPC3 data

In [ ]:
# upload AD2/DNM2/N-WASP data, but only AP2/DNM2 tracking for now

# 8/3/18

tracks_180803ADWCell001_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_001/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell001AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180803ADWCell002_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_002/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell002AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180803ADWCell003_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_003/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell003AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180803ADWCell004_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_004/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell004AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180803ADWCell005_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_005/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell005AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180803ADWCell006_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_006/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell006AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')

# 7/18/18

tracks_180718ADWCell001_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_001/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell001AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180718ADWCell002_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_002/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell002AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')
tracks_180718ADWCell003_1s=display_tracks.load_tracks(unique_user_path_tracks + '/ap2dynm2arcp3_project/TIRF movies revised/tracking_data_test_cyna/180803_ADW/split_channel_data/180803_ADW_003/tracking_settings_gaussian_psf_model_trackinggaplength_2_trackingradius_3_6/180803ADWCell003AP2DNM2_1s/Ch1/Tracking/ProcessedTracks.mat')

In [ ]:
all_tracks_nwasp = [] # a list of all the track objects; each value is one experiment

all_tracks_nwasp.append(tracks_180803ADWCell001_1s)
all_tracks_nwasp.append(tracks_180803ADWCell002_1s)
all_tracks_nwasp.append(tracks_180803ADWCell003_1s)
all_tracks_nwasp.append(tracks_180803ADWCell004_1s)
all_tracks_nwasp.append(tracks_180803ADWCell005_1s)
all_tracks_nwasp.append(tracks_180803ADWCell006_1s)

all_tracks_nwasp.append(tracks_180718ADWCell001_1s)
all_tracks_nwasp.append(tracks_180718ADWCell002_1s)
all_tracks_nwasp.append(tracks_180718ADWCell003_1s)

In [ ]:
# extract valid tracks from nwasp cell line movies
valid_tracks_separate_experiments_nwasp = [display_tracks.remove_tracks_by_criteria(track_set, track_category=[1]) for track_set in all_tracks_nwasp]
# merge all valid tracks into one tracks array
merged_all_valid_tracks_nwasp = merge_tools.merge_experiments(valid_tracks_separate_experiments_nwasp,[list(range(len(track_set))) for track_set in valid_tracks_separate_experiments_nwasp])
# the experiment number for 2 color cell line movies (0-8)
experiment_number_nwasp_label = [i for i in range(len(valid_tracks_separate_experiments_nwasp)) for _ in range(len(valid_tracks_separate_experiments_nwasp[i-1]))]
# labels for the two days of imaging
date_of_experiment_nwasp = [180803 if i < 6 else 180718 for i in range(len(valid_tracks_separate_experiments_nwasp)) for _ in range(len(valid_tracks_separate_experiments_nwasp[i-1]))]
# labels for the number of imaging channels for each track
number_of_channels_label_nwasp = [3 for _ in range(len(experiment_number_nwasp_label))]

cell_condition = ['no_treatment' for i in range(len(number_of_channels_label_nwasp))]

In [ ]:
# extract the output of cmeAnalysis' predictions on whether a track is DNM2 positive or negative
significant_dynamin2_cmeAnalysis_prediction_nwasp = []

# an index map for ProcessedTracks.mat attributes for 2 color tracking experiments from cmeAnalysis
index_dictionary = generate_index_dictionary.return_index_dictionary()

for track in merged_all_valid_tracks_nwasp: # iterate through all tracks

    significant_dynamin2 = track[index_dictionary['index_significantSlave']][1]
    significant_dynamin2_cmeAnalysis_prediction_nwasp.append(significant_dynamin2)

In [ ]:
print('total number of valid tracks: ' + str(len(merged_all_valid_tracks_nwasp)))

In [ ]:
all_track_features_nwasp = feature_extraction_with_buffer.TrackFeatures(merged_all_valid_tracks_nwasp) # an instance of a to-be feature matrix of tracks
all_track_features_nwasp.add_features(possible_track_features) # set the features to be extracted
all_track_features_nwasp.extract_features() # extract all features
extracted_features_all_tracks_nwasp = all_track_features_nwasp.feature_matrix # feature matrix for all tracks

In [ ]:
extracted_features_all_tracks_nwasp.shape

In [ ]:
# merge features with labels (experiment number, date, and number of channels)
extracted_features_all_tracks_nwasp = np.array(extracted_features_all_tracks_nwasp)

merged_features_nwasp = np.concatenate((extracted_features_all_tracks_nwasp,
                                  np.array(experiment_number_nwasp_label).reshape(extracted_features_all_tracks_nwasp.shape[0],-1)), axis=-1)
merged_features_nwasp = np.concatenate((merged_features_nwasp,
                                  np.array(number_of_channels_label_nwasp).reshape(merged_features_nwasp.shape[0],-1)), axis=-1)
merged_features_nwasp = np.concatenate((merged_features_nwasp,
                                  np.array(date_of_experiment_nwasp).reshape(merged_features_nwasp.shape[0],-1)), axis=-1)
merged_features_nwasp = np.concatenate((merged_features_nwasp,
                                  np.array(cell_condition).reshape(merged_features_nwasp.shape[0],-1)), axis=-1)
merged_features_nwasp = np.concatenate((merged_features_nwasp,
                                  np.array(significant_dynamin2_cmeAnalysis_prediction_nwasp).reshape(merged_features_nwasp.shape[0],-1)), axis=-1)

In [ ]:
scaled_features_nwasp = scaler.transform(merged_features_nwasp[:,:len(feature_units)]) # scale features to normal distribution, taking into account all previously scaled data
pcs_nwasp = pca_model.transform(scaled_features_nwasp) # find projections of newly scaled data on previous PC axes
gmm_predictions_nwasp = gmm_model.predict(pcs_nwasp) # find gmm cluster assignments using previously fit model

In [ ]:
pcs_nwasp.shape

# run DNM2 positive events through smoothing and single-peak selection

In [ ]:
# get DNM2 positive events
dnm2_positive_events_nwasp = np.array(list(merged_all_valid_tracks_nwasp))[np.nonzero(gmm_predictions_nwasp==index_DNM2positive)[0]]

In [ ]:
len(dnm2_positive_events_nwasp)

In [ ]:
all_dnm2_signal_nwasp = []

for i in range(len(dnm2_positive_events_nwasp)): # stack all DNM2 intensities

    raw_dnm2_intensity_nwasp = list(return_track_attributes.return_track_amplitude_no_buffer_channel(dnm2_positive_events_nwasp,i,1))

    all_dnm2_signal_nwasp.append(raw_dnm2_intensity_nwasp)

In [ ]:
sos = signal.butter(4, 0.2, 'lp', fs=1, output='sos') # low-pass 4-th order Butterworth filter

filtered_amplitudes_nwasp = [] # filtered DNM2 traces per track of interest

for i in range(len(all_dnm2_signal_nwasp)):

    raw_intensity_nwasp = all_dnm2_signal_nwasp[i]
    # add zeros to end to account for phase shift of near-track-end peaks
    filtered_amplitudes_nwasp.append(list(list(signal.sosfilt(sos, raw_intensity_nwasp)) + [0, 0, 0, 0, 0])) 
    
current_param_outputs_nwasp = [] # one-hot encoding of indices of tracks with a single peak (0: multiple peaks)


for i in range(len(filtered_amplitudes_nwasp)): # iterate through all filtered amplitudes
    
    pvals_dnm2 = return_track_attributes.return_pvals_detection_no_buffer(dnm2_positive_events_nwasp, i, 1)
    
    # measure whether there is 1 peak with the specified peak-finding parameters
    if len(signal.find_peaks(filtered_amplitudes_nwasp[i], 
                             distance=best_fit_peak_params[0], 
                             height=best_fit_peak_params[1],
                             width=best_fit_peak_params[2])[0])==1 and len(np.where(np.array(pvals_dnm2)<0.01)[0])>0:

        current_param_outputs_nwasp.append(1)

    else:

        current_param_outputs_nwasp.append(0)

In [ ]:
len(current_param_outputs_nwasp)

In [ ]:
len(np.where(np.array(current_param_outputs_nwasp)==1)[0])

In [ ]:
data_add_pc_gmm_dataframe_nwasp = np.hstack((pcs_nwasp, gmm_predictions_nwasp.reshape(pcs_nwasp.shape[0], 1)))

In [ ]:
data_add_pc_gmm_dataframe_nwasp.shape

In [ ]:
df_nwasp_data_pcs_gmm_clusters = pd.DataFrame(data_add_pc_gmm_dataframe_nwasp, columns=df_pcs_normal_scaled_with_gmm_cluster.columns)

In [ ]:
df_nwasp_data_pcs_gmm_clusters

In [ ]:
df_nwasp_data_merged_features = pd.DataFrame(merged_features_nwasp, columns=df_merged_features.columns)

In [ ]:
# save the dataframe for ARPC3 tracking
compression_opts = dict(method='zip',
                        archive_name=unique_user_saved_outputs+'/dataframes/df_nwasp_data_pcs_gmm_clusters.csv')  

df_nwasp_data_pcs_gmm_clusters.to_csv(unique_user_saved_outputs+'/dataframes/df_nwasp_data_pcs_gmm_clusters.zip', index=False,
                                                compression=compression_opts) 

In [ ]:
compression_opts = dict(method='zip',
                        archive_name=unique_user_saved_outputs+'/dataframes/df_nwasp_data_merged_features.csv')  

df_nwasp_data_merged_features.to_csv(unique_user_saved_outputs+'/dataframes/df_nwasp_data_merged_features.zip', index=False,
                                                 compression=compression_opts) 

In [ ]:
df_nwasp_data_merged_features

# save N-WASP ccp/hotspot predictions

In [ ]:
np.save(unique_user_saved_outputs+'/dataframes/nwasp_ccp_predictions', current_param_outputs_nwasp)

In [ ]:
len(current_param_outputs_nwasp)

# save all N-WASP valid tracks

In [ ]:
split_valid_tracks_nwasp = np.array_split(np.array(list(merged_all_valid_tracks_nwasp)),number_of_track_splits)

In [ ]:
# save each track array chunk
for i in range(len(split_valid_tracks_nwasp)):

    np.save(unique_user_saved_outputs+"/dataframes/merged_all_valid_nwasp_tracks_"+str(i), split_valid_tracks_nwasp[i])